# Serverless Backend Setup using AWS Lambda

## Imports

In [ ]:
import json
import os
import requests
import sys
sys.path.append("../")

from app_gradio import app
from question_answer.answer import Pipeline

## Build container image

In [ ]:
os.environ["LAMBDA_NAME"] = "admirer-backend"

In [ ]:
!docker build -t $LAMBDA_NAME . --file api_serverless/Dockerfile #--no-cache

In [ ]:
# export LAMBDA_NAME=admirer-backend
# docker run -p 9000:8080 $LAMBDA_NAME\:latest

In [ ]:
!curl -XPOST \
  "http://localhost:9000/2015-03-31/functions/function/invocations" \
  -d '{"image_url": "../question_answer/tests/support/images/img.jpg", "question": "What color is my hair"}'

## Upload to the container registry

In [ ]:
# aws configure

In [ ]:
aws_account_id, = !aws sts get-caller-identity \
  --query "Account"
aws_region, = !aws configure get region 

os.environ["AWS_REGION"] = aws_region
os.environ["AWS_ACCOUNT_ID"] = aws_account_id.strip('"')

!echo $AWS_ACCOUNT_ID
!echo $AWS_REGION

In [ ]:
os.environ["ECR_URI"] = ".".join(
    [os.environ["AWS_ACCOUNT_ID"], "dkr", "ecr", os.environ["AWS_REGION"], "amazonaws.com"])

!echo $ECR_URI

In [ ]:
!aws ecr get-login-password --region $AWS_REGION \
  | docker login --username AWS --password-stdin $ECR_URI

In [ ]:
!aws ecr create-repository \
  --repository-name $LAMBDA_NAME \
  --image-scanning-configuration scanOnPush=true --image-tag-mutability MUTABLE \
  | jq -C

In [ ]:
os.environ["IMAGE_URI"] = "/".join([os.environ["ECR_URI"], os.environ["LAMBDA_NAME"]])

In [ ]:
!docker tag $LAMBDA_NAME\:latest $IMAGE_URI\:latest

In [ ]:
!docker push $IMAGE_URI\:latest

## Create a Lambda function

In [ ]:
os.environ["LAMBDA_ROLE_NAME"] = "lambda-role"

In [ ]:
!aws iam create-role \
  --role-name $LAMBDA_ROLE_NAME \
  --assume-role-policy-document '{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]}' \
  | jq -C

In [ ]:
lambda_role_arn, = !aws iam get-role --role-name $LAMBDA_ROLE_NAME --output json | jq -r '.Role.Arn'
lambda_role_arn = lambda_role_arn.strip('"')

os.environ["LAMBDA_ROLE_ARN"] = lambda_role_arn
!echo $LAMBDA_ROLE_ARN

In [ ]:
# allow this IAM role to execute Lambdas
!aws iam attach-role-policy \
  --role-name $LAMBDA_ROLE_NAME \
  --policy-arn arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole

In [ ]:
# allow this IAM role to write to logs -- required and also important for debugging Lambdas
!aws iam attach-role-policy \
  --role-name $LAMBDA_ROLE_NAME \
  --policy-arn arn:aws:iam::aws:policy/AWSXRayDaemonWriteAccess

In [ ]:
!aws lambda create-function \
  --function-name $LAMBDA_NAME \
  --region $AWS_REGION \
  --package-type Image \
  --code ImageUri=$IMAGE_URI:latest \
  --role $LAMBDA_ROLE_ARN | jq -C

In [ ]:
!aws lambda update-function-configuration \
   --function-name $LAMBDA_NAME \
   --region $AWS_REGION \
   --timeout 60 \
   --memory-size 10240 | jq -C

In [ ]:
!aws lambda invoke \
  --function-name $LAMBDA_NAME \
  --invocation-type RequestResponse \
  --payload '{"image_url": "../question_answer/tests/support/images/img.jpg", "question": "What color is my hair"}' \
  --cli-binary-format raw-in-base64-out lambda.out | jq -C

!cat lambda.out

## Add an HTTP endpoint with a URL

In [ ]:
!aws lambda create-function-url-config \
  --function-name $LAMBDA_NAME \
  --auth-type NONE \
  --cors '{"AllowOrigins": ["*"], "AllowCredentials": false}' \
  | jq -C

In [ ]:
# Careful here!!!
# """
!aws lambda add-permission \
 --function-name $LAMBDA_NAME \
 --action lambda:invokeFunctionUrl \
 --statement-id "open-access" \
 --principal "*" \
 --function-url-auth-type NONE | jq -C
# """

In [ ]:
lambda_url, = !aws lambda get-function-url-config --function-name $LAMBDA_NAME | jq .FunctionUrl
lambda_url = lambda_url.strip('"')

lambda_url

In [ ]:
image_url = "../question_answer/tests/support/images/img.jpg"
question_path = "../question_answer/tests/support/questions/question.txt"
with open(question_path, "r") as f: question = f.readline()

headers = {"Content-type": "application/json"}
payload = json.dumps({"image_url": image_url, "question": question})

response = requests.post(
  lambda_url, data=payload, headers=headers)
pred = response.json()["pred"]

print(pred)

## Connect AWS with Gradio

In [ ]:
serverless_backend = app.PredictorBackend(url=lambda_url)

In [ ]:
frontend_serverless_backend = app.make_frontend(serverless_backend.run, flagging=True)
frontend_serverless_backend.launch(share=True)

In [ ]:
frontend_serverless_backend.close()